### Build and test a Naive Bayes classifier.

We will again use the iris data. 

Goals for this notebook:
1. Understand NB well enough to make a prediction by hand
2. Use the naive_bayes module in scikit-learn
3. First glimpse at the pandas package for manipulating data

In [ ]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.cross_validation import train_test_split

In [ ]:
# Load the data, which is included in sklearn.
iris = load_iris()
print 'Iris target names:', iris.target_names
print 'Iris feature names:', iris.feature_names
X, y = iris.data, iris.target

In [ ]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

## EDA

The iris feature values are real valued measurements in centimeters. Let's look at histograms of each feature.

In [ ]:
# Create a new figure and set the figsize argument so we get square-ish plots of the 4 features.
plt.figure(figsize=(15, 3))

# Iterate over the features, creating a subplot with a histogram for each one.
for feature in range(X_train.shape[1]):
    plt.subplot(1, 4, feature+1)
    plt.hist(X_train[:,feature], 20)
    plt.title(iris.feature_names[feature])

To make things simple, let's binarize these feature values. That is, we'll treat each measurement as either "short" or "long". I'm just going to choose a threshold for each feature.

In [59]:
# Define a function that applies a threshold to turn real valued iris features into 0/1 features.
# 0 will mean "short" and 1 will mean "long".
def binarize_iris(data, thresholds=[6.0, 3.0, 2.5, 1.0]):
    # Initialize a new feature array with the same shape as the original data.
    binarized_data = np.zeros(data.shape)

    # Apply a threshold  to each feature.
    for feature in range(data.shape[1]):
        binarized_data[:,feature] = data[:,feature] > thresholds[feature]
    return binarized_data

# Create new binarized training and test data
binarized_train_data = binarize_iris(X_train)
binarized_test_data = binarize_iris(X_test)

print X_train[:10, ]
print binarized_train_data[:10, ]

[[ 5.9  3.   4.2  1.5]
 [ 5.8  2.6  4.   1.2]
 [ 6.8  3.   5.5  2.1]
 [ 4.7  3.2  1.3  0.2]
 [ 6.9  3.1  5.1  2.3]
 [ 5.   3.5  1.6  0.6]
 [ 5.4  3.7  1.5  0.2]
 [ 5.   2.   3.5  1. ]
 [ 6.5  3.   5.5  1.8]
 [ 6.7  3.3  5.7  2.5]]
[[ 0.  0.  1.  1.]
 [ 0.  0.  1.  1.]
 [ 1.  0.  1.  1.]
 [ 0.  1.  0.  0.]
 [ 1.  1.  1.  1.]
 [ 0.  1.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  1.  1.]
 [ 1.  1.  1.  1.]]


Recall that Naive Bayes assumes conditional independence of features. With $Y$ the set of labels and $X$ the set of features ($y$ is a specific label and $x$ is a specific feature), Naive Bayes gives the probability of a label $y$ given input features $X$ as:

$ \displaystyle P(y|X) \approx 
  \frac { P(y) \prod_{x \in X} P(x|y) }
        { \sum_{y \in Y} P(y) \prod_{x \in X} P(x|y) }
$

Let's estimate some of these probabilities using maximum likelihood, which is just a matter of counting and normalizing. We'll start with the prior probability of the label $P(y)$.

In [ ]:
# Initialize counters for all labels to zero.
label_counts = [0 for i in iris.target_names]

# Iterate over labels in the training data and update counts.
for label in y_train:
    label_counts[label] += 1

# Normalize counts to get a probability distribution.
total = sum(label_counts)
label_probs = [1.0 * count / total for count in label_counts]
for (prob, name) in zip(label_probs, iris.target_names):
    print '%15s : %.2f' %(name, prob)

### Repeat above three cells using pandas

Pandas allows us to stop thinking about code in terms of procedures, and start thinking about code in terms of _data manipulation_.

- Check out pandas here: pandas.pydata.org
- [Here](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf) is cheat sheet to get a sense for what is possible.
- [This](http://pandas.pydata.org/pandas-docs/stable/10min.html) is probably the best place to get started


Load the training data into a pandas data frame

In [ ]:
iris.feature_names

In [ ]:
colnames = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
df = pd.DataFrame(X, columns=colnames)
df.head()

In [ ]:
target = pd.Series(y)
df['target'] = target
df.head()

In [ ]:
df = df.replace({'target': {0: 'setosa', 1: 'versicolor', 2: 'virginica'}})
df.head()

In [ ]:
train_df, test_df = train_test_split(df, random_state=0)

Plot some histograms

In [ ]:
_ = train_df.hist(bins=20)

Binarize the columns

In [ ]:
thresholds = np.array([6, 3, 2.5, 1])
train_df_binarized = train_df[colnames].apply(lambda row: row > thresholds, axis=1)
test_df_binarized = test_df[colnames].apply(lambda row: row > thresholds, axis=1)
train_df_binarized.head()

Estimate priors

In [ ]:
train_df.target.value_counts(normalize=True)

## Naive Bayes by Hand

Next, let's estimate $P(X|Y)$, that is, the probability of each feature given each label. Remember that we can get the conditional probability from the joint distribution:

$\displaystyle P(X|Y) = \frac{ P(X,Y) } { P(Y) } \approx \frac{ \textrm{Count}(X,Y) } { \textrm{Count}(Y) }$

Let's think carefully about the size of the count matrix we need to collect. There are 3 labels $y_1$, $y_2$, and $y_3$ and 4 features $x_0$, $x_1$, $x_2$, and $x_3$. Each feature has 2 possible values, 0 or 1. So there are actually $4 \times 2 \times 3=24$ probabilities we need to estimate: 

$P(x_0=0, Y=y_0)$

$P(x_0=1, Y=y_0)$

$P(x_1=0, Y=y_0)$

$P(x_1=1, Y=y_0)$

...

However, we already estimated (above) the probability of each label. And, we know that each feature value is either 0 or 1. So, for example,

$P(x_0=0, Y=\textrm{setosa}) + P(x_0=1, Y=\textrm{setosa}) = P(Y=\textrm{setosa}) \approx 0.31$.

As a result, we can just estimate probabilities for one of the feature values, say, $x_i = 1$. This requires a $4 \times 3$ matrix.

In [ ]:
train_df_binarized['target'] = train_df['target']
train_df_binarized.head()

In [ ]:
conditional_probs = train_df_binarized.groupby('target').mean()
conditional_probs

That is P(sepal length > 1 | versicolor) = .35

In [ ]:
priors = train_df_binarized['target'].value_counts(normalize=True)
priors

Now that we have all the pieces, let's try making a prediction for the first test example. 

In [ ]:
test_instance = test_df.iloc[0]
test_instance

In [ ]:
test_instance_binarized = test_df_binarized.iloc[0]
test_instance_binarized

#### Exercise

Compute the probability, according to the Naive Bayes model, that the label of this observation is 'virginica'

1: Write a function that takes a label as input and returns _the numerator_ of the NB equation. That is, P(y) * P(x | y)

Note: This function does not need to handle any possible input observation. It can be hard-coded to specifically handle `test_instance_binarized`.

In [ ]:
def lognum(label):
    p_label = priors[label]
    p_sl = 1 - conditional_probs.loc[label, 'sepal_length']
    p_sw = 1 - conditional_probs.loc[label, 'sepal_width']
    p_pl = conditional_probs.loc[label, 'petal_length']
    p_pw = conditional_probs.loc[label, 'petal_width']
    return np.sum(np.log([p_label, p_sl, p_sw, p_pl, p_pw]))

Test

In [ ]:
label = 'virginica'
lognum('virginica')

2: Compute the log of the numberator for each possible label

In [ ]:
lognums = pd.Series({label: lognum(label) for label in ('virginica', 'setosa', 'versicolor')})
lognums

3: Convert the results of (2) into probabilities

In [ ]:
lognums.apply(np.exp) / lognums.apply(np.exp).sum()

## Naive Bayes in Scikit-Learn

In [ ]:
print train_df_binarized.head()
print test_df_binarized.head()

### BernoulliNB means _the features are distributed Bernoulli_

In [ ]:
from sklearn.naive_bayes import BernoulliNB
nb = BernoulliNB(alpha=1)
nb.fit(train_df_binarized[colnames], train_df.target)

predicted_probabilities = nb.predict_proba(test_df_binarized[colnames])
pred = nb.predict(test_df_binarized[colnames])

correct = pred == test_df['target']
accuracy = np.mean(correct)
accuracy

In [ ]:
nb.score(test_df_binarized[colnames], test_df['target'])

In [ ]:
pred

In [ ]:
pd.DataFrame(predicted_probabilities, columns=['setosa', 'versicolor', 'virginica'])

### GaussianNB _means the features are distributed Gaussian_

Exercise: Build a GaussianNB model using sklearn, and estimate the accuracy on the test set.

In [ ]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()
nb.fit(train_df[colnames], train_df['target'])
nb.score(test_df[colnames], test_df['target'])

Let's look at those predicted probabilities

In [ ]:
predicted_probabilities = pd.DataFrame(nb.predict_proba(test_df[colnames]), 
                                       columns=['setosa', 'versicolor', 'virginica'])
predicted_probabilities.head()

The predicted probabilities are _very_ confident. Since the test predictions are also extremely accurate, we might conclude that GaussianNB is an appropriate model

### MultinomialNB _means the features are distributed multinomial_

In [60]:
cuts = [0, .2, .4, .6, .8, 1]
breaks = train_df[colnames].quantile(cuts)
breaks.loc[0, :] = 0  # Set a global minimimum
breaks.loc[1, :] = 10  # Set a global maximum
print breaks

     sepal_length  sepal_width  petal_length  petal_width
0.0          0.00          0.0          0.00         0.00
0.2          5.02          2.7          1.50         0.20
0.4          5.60          3.0          3.90         1.14
0.6          6.16          3.1          4.70         1.50
0.8          6.70          3.4          5.48         2.00
1.0         10.00         10.0         10.00        10.00


In [ ]:
# Discretize each column according to the breaks defined above
train_df_mult = train_df[colnames].apply(
    lambda col: pd.cut(col, bins=breaks[col.name], include_lowest=True, labels=False)
)
test_df_mult = test_df[colnames].apply(
    lambda col: pd.cut(col, bins=breaks[col.name], include_lowest=True, labels=False))

In [ ]:
train_df_mult.head()

In [ ]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB(alpha=1)
nb.fit(train_df_mult, train_df['target'])
nb.score(test_df_mult, test_df['target'])

## Bonus:

Consider the following threshold possibilities for each of the four features.

```
sepal_length_thresh = [5.0, 5.6, 6.2, 6.5]
sepal_width_thresh = [2.7, 3.0, 3.2, 3.4]
petal_length_thresh = [2, 3, 4, 5]
petal_width_thresh = [.5, 1, 1.5, 2]
```

Find the combination of thresholds (one from each list), that gives the BernoulliNB model with the highest accuracy.

Use a train/test/dev split, and report accuracy on the test set.